# run_car

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
dyn_plot = True
class DynamicUpdate():
    def on_launch(self):
        #Set up plot
        self.figure, self.ax = plt.subplots()
        self.lines, = self.ax.plot([],[])
        #Autoscale on unknown axis and known lims on the other
        self.ax.set_autoscaley_on(True)
        self.ax.set_autoscalex_on(True)
        #Other stuff
        self.ax.grid()

    def on_running(self, xdata, ydata):
        #Update data (with the new _and_ the old points)
        self.lines.set_xdata(xdata)
        self.lines.set_ydata(ydata)
        #Need both of these in order to rescale
        self.ax.relim()
        self.ax.autoscale_view()
        #We need to draw *and* flush
        self.figure.canvas.draw()
        self.figure.canvas.flush_events()

In [2]:
import datetime
import numpy as np
import random

In [3]:
from cars.world import SimpleCarWorld
from cars.agent import SimpleCarAgent
from cars.physics import SimplePhysics
from cars.track import generate_map

In [4]:
from tqdm import tqdm_notebook

In [5]:
from functools import reduce

In [6]:
import pandas as pd

In [7]:
def create_map(seed, agent):
    np.random.seed(seed)
    random.seed(seed)
    m = generate_map(8, 5, 3, 3)
    w = SimpleCarWorld([agent], m, SimplePhysics, None, timedelta=0.2)
    return w

In [8]:
import seaborn as sns

# Параметры

In [9]:
map_seeds = [3, 13, 23]

In [10]:
# За сколько шагов модель начинает учитывать вознаграждение
map_seed = map_seeds[0]

# Нарабатываем данные, учимся и оцениваемся

In [11]:
log_list = []

In [12]:
def norm_y(y):
    return y / 15.0

In [13]:
def mine_data(agent, action_trainer_params,
              clear_history, map_seed, steps,
              epochs, mini_batch_size, eta, reward_shift, alpha):
    agent.action_trainer.set_params(**action_trainer_params)
    if clear_history:
        agent.clear_history()
    
    # сбрасываем статистику
    agent.action_trainer.reset_steps()
    
    #mine data
    w = create_map(seed=map_seed, agent=agent)
    w.run(steps=tqdm_notebook(range(steps), desc="train", leave=False), visual=False, save=False)

    steps_str = "total: {:d}, bad: {:.4f}, good: {:.4f}, diff: {:.4f}, other: {:.4f}".format(
        agent.action_trainer.steps[0],
        agent.action_trainer.steps[1] / agent.action_trainer.steps[0],
        agent.action_trainer.steps[2] / agent.action_trainer.steps[0],
        agent.action_trainer.steps[3] / agent.action_trainer.steps[0],
        1 - sum(agent.action_trainer.steps[1:]) / agent.action_trainer.steps[0])
    
    # prepare train data
    X_train = np.concatenate([agent.sensor_data_history, agent.chosen_actions_history], axis=1)
    y_train = np.array(agent.reward_history)
    # сглаживаем пики, чтобы сеть небольшая сеть могла дать адекватное предсказание
    mean_train_revard = y_train.mean()
    y_train = norm_y(y_train)
    
    y_train = np.pad(y_train, (0, reward_shift + 1), mode="constant")
    y_train = reduce(lambda a, b: a + b, [y_train[i:(i - reward_shift - 1)] * (alpha ** (i - 1))
                                          for i in range(1, reward_shift+1)])

    #train NN
    train_data = [(x[:, np.newaxis], y) for x, y in zip(X_train, y_train)]
    train_rmse_before = agent.neural_net.evaluate(X_train.T, y_train)
    next(agent.neural_net.SGD(training_data=train_data,
                         epochs=epochs,
                         mini_batch_size=mini_batch_size,
                         eta=eta))
    train_rmse_after = agent.neural_net.evaluate(X_train.T, y_train)

    collisions = sum((1 if x < 0 and  0 < len(agent.reward_history) - i < steps else 0 
                      for i, x in enumerate(agent.reward_history))) / steps
    
    # evaluate
    w = create_map(seed=map_seed, agent=agent)
    mean_test_reward = w.evaluate_agent(agent, steps=tqdm_notebook(range(800), desc="test", leave=False), visual=False)
    
    
    log_message = f"""map_seed = {map_seed}
train_rmse_before = {train_rmse_before:.9f}, train_rmse_after = {train_rmse_after:.9f}, 
mean_train_revard = {mean_train_revard:.3f}, mean_test_reward  = {mean_test_reward:.3f},
steps = {steps_str},
collisions = {collisions}"""

    log_list.append(log_message)
    if not dyn_plot:
        chart_count = agent.neural_net.num_layers
        plt.figure(figsize=(5 * chart_count, 2))
        for l in range(chart_count - 1):
            plt.subplot(1, chart_count, l+1)
            ax = sns.heatmap(agent.neural_net.weights[l])

        plt.subplot(1, chart_count, chart_count)    
        plt.text(0.05, 0.95, log_message, size=12, ha='left', va='top', family='monospace')

        plt.show()
    else:
        print(log_message)

In [14]:
kvparams = {
    "clear_history": False,
    "map_seed": map_seed,
    "epochs": 100,
    "mini_batch_size": 50,
    "eta": 0.5,
    "reward_shift": 4,
    "alpha": 0.7
}

act_trainer = {
    "low": 0.0,
    "high": 0.0,
    "random_default": 0.8,
    "random_bad": 0.8,
    "random_good": 0.8,
    "random_diff": 0.8,
    "bad_choice_p": 0.01
}

random.seed(42)
np.random.seed(42)
agent1 = SimpleCarAgent(name="Ivan", history_data=20000)

In [15]:
print("\n\n".join(log_list) + "\n----\n")
for _ in tqdm_notebook(range(1), desc="main"):
    mine_data(agent1, act_trainer, steps=4000, **kvparams)


----

map_seed = 3
train_rmse_before = 2.171822012, train_rmse_after = 0.162078707, 
mean_train_revard = -0.459, mean_test_reward  = -0.844,
steps = total: 4000, bad: 1.0000, good: 0.0000, diff: 0.0000, other: 0.0000,
collisions = 0.06025



In [16]:
y_train_setting = norm_y(np.array(agent1.reward_history))

act_trainer["low"] = -0.4
act_trainer["high"] = -0.1

act_trainer["random_default"] = 0.01
act_trainer["random_bad"] = 0.80
act_trainer["random_good"] = 0.01
act_trainer["random_diff"] = 0.01

act_trainer["bad_choice_p"] = 0.01

In [17]:
print("\n\n".join(log_list) + "\n----\n")

for _ in tqdm_notebook(range(5), desc="main"):
    mine_data(agent1, act_trainer, steps=2000, **kvparams)
    print("")

map_seed = 3
train_rmse_before = 2.171822012, train_rmse_after = 0.162078707, 
mean_train_revard = -0.459, mean_test_reward  = -0.844,
steps = total: 4000, bad: 1.0000, good: 0.0000, diff: 0.0000, other: 0.0000,
collisions = 0.06025
----

map_seed = 3
train_rmse_before = 0.451088001, train_rmse_after = 0.150979906, 
mean_train_revard = -1.627, mean_test_reward  = -0.290,
steps = total: 2000, bad: 0.0010, good: 0.6135, diff: 0.0000, other: 0.3855,
collisions = 0.669

map_seed = 3
train_rmse_before = 0.288060070, train_rmse_after = 0.136973713, 
mean_train_revard = -1.567, mean_test_reward  = -0.286,
steps = total: 2000, bad: 0.0085, good: 0.7680, diff: 0.0020, other: 0.2215,
collisions = 0.2905

map_seed = 3
train_rmse_before = 0.140584161, train_rmse_after = 0.125163028, 
mean_train_revard = -1.288, mean_test_reward  = -0.480,
steps = total: 2000, bad: 0.0240, good: 0.7110, diff: 0.0055, other: 0.2595,
collisions = 0.061

map_seed = 3
train_rmse_before = 0.139857314, train_rmse_after =

In [18]:
kvparams["eta"] = 0.1

act_trainer["low"] = -0.3
act_trainer["high"] = -0.1

act_trainer["random_bad"] = 0.99

In [19]:
print("\n\n".join(log_list) + "\n----\n")

for _ in tqdm_notebook(range(5), desc="main"):
    mine_data(agent1, act_trainer, steps=5000, **kvparams)
    print("")

map_seed = 3
train_rmse_before = 2.171822012, train_rmse_after = 0.162078707, 
mean_train_revard = -0.459, mean_test_reward  = -0.844,
steps = total: 4000, bad: 1.0000, good: 0.0000, diff: 0.0000, other: 0.0000,
collisions = 0.06025

map_seed = 3
train_rmse_before = 0.451088001, train_rmse_after = 0.150979906, 
mean_train_revard = -1.627, mean_test_reward  = -0.290,
steps = total: 2000, bad: 0.0010, good: 0.6135, diff: 0.0000, other: 0.3855,
collisions = 0.669

map_seed = 3
train_rmse_before = 0.288060070, train_rmse_after = 0.136973713, 
mean_train_revard = -1.567, mean_test_reward  = -0.286,
steps = total: 2000, bad: 0.0085, good: 0.7680, diff: 0.0020, other: 0.2215,
collisions = 0.2905

map_seed = 3
train_rmse_before = 0.140584161, train_rmse_after = 0.125163028, 
mean_train_revard = -1.288, mean_test_reward  = -0.480,
steps = total: 2000, bad: 0.0240, good: 0.7110, diff: 0.0055, other: 0.2595,
collisions = 0.061

map_seed = 3
train_rmse_before = 0.139857314, train_rmse_after = 0.13

# Смотрим вживую

In [ ]:
def visualize(agent, map_seed):
    agent.evaluate_mode=True
    print("visualize on map = {}".format(map_seed))
    # agent.clear_history()
    w = create_map(seed=map_seed, agent=agent)
    w.run(steps=None, visual=True, save=False)

In [ ]:
visualize(agent1, map_seed)

# Отладка

In [ ]:
raise Exception("Здесь отладка")

In [ ]:
agent_dbg = agent1

In [ ]:
agent_dbg.clear_history()
#mine data
w = create_map(seed=map_seed, agent=agent_dbg)
w.run(steps=tqdm_notebook(range(10000), desc="train", leave=False), visual=False, save=False)

In [ ]:
# prepare train data
X_train = np.concatenate([agent_dbg.sensor_data_history, agent_dbg.chosen_actions_history], axis=1)
y_train = np.array(agent_dbg.reward_history)
mean_train_revard = y_train.mean()
y_train = norm_y(y_train)
y_clean = y_train.copy()

mean_train_revard

In [ ]:
reward_shift = 6
alpha = 0.8

In [ ]:
y_train = np.pad(y_train, (0, 2 * reward_shift), mode="constant")
y_train = reduce(lambda a, b: a + b, [y_train[i:(-2 * reward_shift + i)] * (alpha ** (i - 1))
                                      for i in range(1, reward_shift+1)])

In [ ]:
y_predict = agent_dbg.neural_net.feedforward(X_train.T)

In [ ]:
pd_visual = pd.DataFrame(X_train,
                         columns=["velocity", "angle"] +
                         ["lid{}".format(i) for i in range(agent_dbg.rays)] +
                         ["steering", "acceleration"])
pd_visual["y_clean"] = y_clean
pd_visual["y_train"] = y_train
pd_visual["y_predict"] = y_predict.flatten()

In [ ]:
(pd_visual.y_clean[200:] < 0).argmax()

In [ ]:
pd_visual.loc[265:285]

In [ ]:
from cars.utils import Action

In [ ]:
sensor_info = pd_visual.loc[273].values[:7]
rewards_to_controls_map = []
# дискретизируем множество значений, так как все возможные мы точно предсказать не сможем
for steering in np.linspace(-1, 1, 3):  # выбирать можно и другую частоту дискретизации, но
    for acceleration in np.linspace(-0.75, 0.75, 3):  # в наших тестах будет именно такая
        action = Action(steering, acceleration)
        agent_vector_representation = np.append(sensor_info, action)
        agent_vector_representation = agent_vector_representation.flatten()[:, np.newaxis]
        predicted_reward = float(agent_dbg.neural_net.feedforward(agent_vector_representation))
        rewards_to_controls_map.append((predicted_reward, action))

rewards_to_controls_map

In [ ]:
sorted(rewards_to_controls_map, key=lambda x: x[0])[-1]

In [ ]:
sensor_info = pd_visual.loc[275].values[:7]
rewards_to_controls_map = []
# дискретизируем множество значений, так как все возможные мы точно предсказать не сможем
for steering in np.linspace(-1, 1, 3):  # выбирать можно и другую частоту дискретизации, но
    for acceleration in np.linspace(-0.75, 0.75, 3):  # в наших тестах будет именно такая
        action = Action(steering, acceleration)
        agent_vector_representation = np.append(sensor_info, action)
        agent_vector_representation = agent_vector_representation.flatten()[:, np.newaxis]
        predicted_reward = float(agent_dbg.neural_net.feedforward(agent_vector_representation))
        rewards_to_controls_map.append((predicted_reward, action))

rewards_to_controls_map

In [ ]:
sorted(rewards_to_controls_map, key=lambda x: x[0])[-1]

In [ ]:
plt.ion()

In [ ]:
pd_visual[-8000:-6000].plot(y=["y_clean", "y_train", "y_predict"], figsize=(64, 4))

In [ ]:
rmse_before = ((pd_visual.y_train - pd_visual.y_predict)**2).mean() ** 0.5
rmse_before

In [ ]:
train_data = [(x[:, np.newaxis], y) for x, y in zip(X_train, y_train)]

In [ ]:
from learning_algorithms.network import Network
agent_dbg.neural_net = Network([9, 18, 8, 1], output_function=lambda x: x, output_derivative=lambda x: 1)

In [ ]:
epoch_num = 1
dyp_x = []
dyp_y = []

In [ ]:
plt.ion()
dyp = DynamicUpdate()
dyp.on_launch()

try:
    for res in agent_dbg.neural_net.SGD(
            training_data=train_data,
            epochs=500,
            mini_batch_size=50,
            eta=0.1,
            test_data=(X_train.T, y_train)):
        dyp_x.append(epoch_num)
        epoch_num += 1
        dyp_y.append(res)
        if epoch_num % 10 == 0:
            dyp.on_running(dyp_x, dyp_y)
except KeyboardInterrupt:
    print("min rmse: {}".format(min(dyp_y)))
print("min rmse: {}".format(min(dyp_y)))

In [ ]:

              epochs, mini_batch_size, eta, reward_shift, alpha):

    




    # evaluate
    w = create_map(seed=map_seed, agent=agent)
    mean_test_reward = w.evaluate_agent(agent, steps=tqdm_notebook(range(800), desc="test", leave=False), visual=False)
    log_message = f"""map_seed = {map_seed}
agent_random_action = {agent_random_action:.2f}
train_rmse = {train_rmse:.9f}
mean_train_revard = {mean_train_revard:.3f}
mean_test_reward  = {mean_test_reward:.3f}"""

    chart_count = agent.neural_net.num_layers
    plt.figure(figsize=(5 * chart_count, 2))
    for l in range(chart_count - 1):
        plt.subplot(1, chart_count, l+1)
        ax = sns.heatmap(agent.neural_net.weights[l])
        
    plt.subplot(1, chart_count, chart_count)    
    plt.text(0.05, 0.95, log_message, size=12, ha='left', va='top', family='monospace')
    
    plt.show()